In [1]:
from astropy.table import Table
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy import optimize
import math
import random
from condensation_temperature import *
from tqdm import tqdm
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
t= Table.read('solar_twins_data.fits') #fits file as table 

exclusions = ['HIP19911', 'HIP108158', 'HIP109821', 'HIP115577', 'HIP14501', 'HIP28066', 'HIP30476',
              'HIP33094', 'HIP65708', 'HIP73241', 'HIP74432', 'HIP64150']
for u in exclusions:
    for i, txt in enumerate(t['star_name']):
        if txt == u:
            t.remove_row(i)

for i, words in enumerate(t['Fe']):
   t['Fe'][i] = 0

t['O'][8] = 0
t['O_err'][8] = 10**6

In [3]:
def delta(param, x, y): # Hogg 2010 eqn 30 : ∆i = (vˆT Zi) − b cos θ   
    (m,b) = param
    theta = np.arctan(m)
    v = np.array([- np.sin(theta), np.cos(theta)]) # v = [− sin θ  cos θ]   

    disp = np.zeros_like(y)
    for i, (ys, xs) in enumerate(zip(y, x)):
        z0 = np.asarray([0.0, b])
        zi = np.asarray([xs, ys])
        disp[i] = np.dot( v, zi - z0 )
    return disp

#2D uncertainties: lnL = K -  sum [delta^2 / 2* sum^2]  
def twodnlnL(param, x, y, errx, erry):
    K = 0
    delt = delta(param, x, y)
    (m,b) = param

    #Σ^2 = vˆT Si v  
    theta = np.arctan(m)
    v = np.array([- np.sin(theta), np.cos(theta)]) # v = [− sin θ,  cos θ]
    
    var = np.zeros_like(erry)
    for i, (dy, dx) in enumerate(zip(erry, errx)):
        cov = np.eye(2)
        cov[0,0] = dx**2
        cov[1,1] = dy**2 
        var[i] = np.dot(v.T, np.dot(cov, v))
        
    sigmasq = var 
    return 0.5 * np.sum(delt**2/sigmasq + np.log(sigmasq))

def find_m_b(x,y,err):
    #C     
    errorsq = np.square(err)
    C = np.diag(errorsq)

    #A 
    xb = ([1] * len(x))
    mata = []
    for z, txt in enumerate(x):
        mata.append(x[z])
        mata.append(xb[z])
    A= np.matrix(mata).reshape((len(x), 2))

    #plugging in 
    At = np.transpose(A)
    invC = np.linalg.inv(C)
    pt1 = np.dot(At, np.dot(invC,A))
    invpt1= np.linalg.inv(pt1)
    pt2 = np.dot(At, np.dot(invC, y)).T
    cov = np.dot(invpt1, pt2)

    m = float(cov[0])
    b = float(cov[1])
    return m ,b 

def residuals(x, y, error):
    mborig = find_m_b(x, y, error)
    m = mborig[0]
    b = mborig[1]

    predicted_values = [] #y values from slope
    pv = 0
    for u in x:
        pv = (m*u) + b
        predicted_values.append(pv)
        pv = 0

    prev = np.array(predicted_values)
    abu = np.array(y)
    diff = abu - prev #difference between slope and measured values  
    return diff

def stdev(x):
    mean = np.mean(x)
    fun = 0
    for c in x: 
        j = (c-mean)**2
        fun = fun + j 
    s = np.sqrt(fun/(len(x)))
    return s

#chi squared, Hogg 2010 eq 7 :  [Y - AX]^T C^-1 [Y - AX] 
def chisquared(param, x, y, erro):
    for h, txt in enumerate(y): #removing nan values
        if (math.isnan(txt) == True):
            del x[h]
            del y[h]
            del erro[h]

    (m,b) = param #for optimization - m and b 
    X = (m,b)
    
    #A 
    ab = ([1] * len(x))
    Amat = []
    for z, txt in enumerate(x):
        Amat.append(x[z])
        Amat.append(ab[z])
    A= np.array(Amat).reshape((len(x), 2))

    #C  
    errorsq = np.square(erro)
    C = np.diag(errorsq)
    invsC = np.linalg.inv(C)

    #plugging in  
    AT= np.transpose(A)
    AX = np.dot(A,X)
    yax = (y - AX)
    yaxT = np.transpose(yax)
    yaxTinvsC = np.dot(yaxT, invsC)

    chisq = (np.dot(yaxTinvsC, yax))
    return (chisq)

In [6]:
elements = []
for n in t.colnames:
    if len(n) < 3 :
        elements.append(n) #list of element names
elements = np.array(elements)

In [7]:
initial_guess = (0.1, 0.2)  
standarddev = []
standarddev_resid = []
    
for l, val in enumerate(elements):
    resid = residuals(t['age'], t[val], t[val + '_err'])
    
    standarddev.append(stdev(t[val]))
    standarddev_resid.append(stdev(resid))

In [8]:
Table([elements, standarddev, standarddev_resid, np.array(standarddev)/np.array(standarddev_resid)], 
      names = ('Element', 'Standard Deviation', 'Residual Standard Deviation', 'Divided'))

Element,Standard Deviation,Residual Standard Deviation,Divided
str2,float64,float64,float64
Fe,0.0,0.0,nan
O,0.033611419319191496,0.026719476522191497,1.2579370442110267
Na,0.03595281172982548,0.029209208399030934,1.2308725124854214
Mg,0.02996596613322483,0.016754876051260633,1.788492259897691
Al,0.038449476465674134,0.0182233174856738,2.109905427258295
Si,0.019251252599340084,0.011891489271426116,1.6189101432062538
S,0.03854360986715821,0.032257028387927114,1.194890285727125
Ca,0.010348971498301865,0.010006651266438779,1.0342092696895704
V,0.012400154871228418,0.012162507827491653,1.0195393127065124
